In [1]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.utils import np_utils
from keras.preprocessing.text import Tokenizer
from keras import metrics

from pprint import pprint
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

C:\Users\geop\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#Load the entire dataset
data = pd.read_csv('../../Results/results_old.csv', header = 0, names = ['Query', 'Job Title', 'Description'])

#Split data to train and test (80 - 20)

train, test = train_test_split(data, test_size = 0.2)

train_descs = train['Description']
train_labels = train['Query']
#train_labels = train['Job Title']
 
test_descs = test['Description']
test_labels = test['Query']
#test_labels = test['Job Title']

print(len(data['Query'].unique().tolist()))
print(len(data['Job Title'].unique().tolist()))

print(len(train_labels.unique().tolist()))
print(len(test_labels.unique().tolist()))

82
1214
82
79


In [3]:
num_labels = len(train_labels.unique().tolist())
vocab_size = 1000
batch_size = 100
nb_epoch = 30

# define Tokenizer with Vocab Size
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(train_descs)
 
x_train = tokenizer.texts_to_matrix(train_descs, mode='tfidf')
x_test = tokenizer.texts_to_matrix(test_descs, mode='tfidf')
 
encoder = LabelBinarizer()
encoder.fit(train_labels)
y_train = encoder.transform(train_labels)
y_test = encoder.transform(test_labels)

In [5]:
model = Sequential()
model.add(Dense(512, input_shape=(vocab_size,)))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(num_labels))
model.add(Activation('softmax'))
model.summary()
 
model.compile(loss = 'categorical_crossentropy',
              optimizer = 'adam', # or 'sgd'
              metrics = [metrics.categorical_accuracy, 'accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               512512    
_________________________________________________________________
activation_1 (Activation)    (None, 512)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
activation_2 (Activation)    (None, 512)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 81)                41553     
__________

In [6]:
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=nb_epoch,
                    verbose=1,
                    validation_split=0.1)

Train on 1080 samples, validate on 120 samples
Epoch 1/30
1080/1080 [==============================] - 1s 537us/step - loss: 4.6311 - categorical_accuracy: 0.0231 - acc: 0.0231 - val_loss: 4.1901 - val_categorical_accuracy: 0.0583 - val_acc: 0.0583
Epoch 2/30
1080/1080 [==============================] - 0s 236us/step - loss: 3.7300 - categorical_accuracy: 0.1815 - acc: 0.1815 - val_loss: 3.9590 - val_categorical_accuracy: 0.1583 - val_acc: 0.1583
Epoch 3/30
1080/1080 [==============================] - 0s 259us/step - loss: 2.9215 - categorical_accuracy: 0.3750 - acc: 0.3750 - val_loss: 3.5483 - val_categorical_accuracy: 0.2000 - val_acc: 0.2000
Epoch 4/30
1080/1080 [==============================] - 0s 243us/step - loss: 2.1094 - categorical_accuracy: 0.5370 - acc: 0.5370 - val_loss: 3.2345 - val_categorical_accuracy: 0.2750 - val_acc: 0.2750
Epoch 5/30
1080/1080 [==============================] - 0s 237us/step - loss: 1.4358 - categorical_accuracy: 0.6648 - acc: 0.6648 - val_loss: 3.1

In [7]:
score = model.evaluate(x_test, y_test,
                       batch_size=batch_size, verbose=1)
 
print('\nTest categorical_crossentropy:', score[0])
print('Categorical accuracy:', score[1])
print('Accuracy:', score[2])


300/300 [==============================] - 0s 70us/step

Test categorical_crossentropy: 3.21578041712443
Categorical accuracy: 0.37666667501131695
Accuracy: 0.37666667501131695
